## Load Libraries

In [33]:
import pandas as pd
import numpy as np
import random as rnd
import datetime

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
#sns.set_style('whitegrid')
%matplotlib inline

# plotly
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import cufflinks as cf
cf.go_offline()

from fastai.tabular.all import *

# SKlearn
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import *
from time import time


from tqdm import tqdm
from joblib import Parallel, delayed
from sklearn.base import clone


## Load Data

In [34]:
filepath = "/Users/Kapil/PycharmProjects/stock-price"
train = pd.read_csv(filepath + "/data/train.csv")
test = pd.read_csv(filepath + "/data/test.csv")

In [35]:
SEED = 13
np.random.seed(SEED)

In [36]:
Y_train = train['Up']
#X_train = train.drop(['Up'], axis = 1)

## Functions
#### Preprocessing function

In [37]:
def LagPreproc(df):
    data = pd.DataFrame()
    data['Today'] = df['Close'].pct_change() * 100
    #data = data.rename("Today")
    for i in range(1,6):
        data['Lag'+str(i)] = data['Today'].shift(i)
    data = data.fillna(data.mean())
    return data

#### Specificity function

In [38]:
def specificity(y_truth, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_truth, y_pred).ravel()

    specificity = tn / (tn+fp)
    return specificity

#### Fit and Score function

In [39]:
def fit_and_score(model, train_index, valid_index, X, Y):
    """
    function to use in a joblib parallel env, takes the inputs and returns metrics
    return: (roc_train, roc_val, f1_train, f1_val)
    """
    # get X and y vectors
    x_train, x_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = Y.iloc[train_index], Y.iloc[valid_index]

    # over sample x
    #x_train, y_train = ros.fit_resample(x_train, y_train)

    # preprocess
#     x_train = ct.fit_transform(x_train)
#     x_valid = ct.transform(x_valid)

    # train model
    model.fit(x_train, y_train)

    # append scores
    y_train_preds = model.predict(x_train)
    y_valid_preds = model.predict(x_valid)

    f1_train = f1_score(y_train, y_train_preds)
    roc_train = roc_auc_score(y_train, model.predict_proba(x_train)[:,1])
    mcc_train = matthews_corrcoef(y_train, y_train_preds)
    acc_train = accuracy_score(y_train, y_train_preds)

    f1_val = f1_score(y_valid, y_valid_preds)
    roc_val = roc_auc_score(y_valid, model.predict_proba(x_valid)[:,1])
    mcc_val = matthews_corrcoef(y_valid, y_valid_preds)
    acc_val= accuracy_score(y_train, y_train_preds)

    log_line = 'Train score: ' + str(acc_train) + ' Validation score: ' + str(roc_val)
    tqdm.write(log_line)

    return roc_train, roc_val, f1_train, f1_val, mcc_train, mcc_val

### Preprocess train and test

In [40]:
X_train  = LagPreproc(train)
X_test = LagPreproc(test)


## Models
#### Logistic Regression

In [41]:
model = LogisticRegression(solver = 'liblinear', C = 0.01, penalty = 'l1')

#### Decision Tree

In [10]:
model = DecisionTreeClassifier(max_depth=10, min_samples_leaf = 30)

#### Random Forest

In [11]:
model = RandomForestClassifier(criterion='gini', n_estimators = 800, max_depth = 18, n_jobs= -1)

#### LightGBM

In [19]:
model = LGBMClassifier(boosting_type = 'gbdt', learning_rate = 0.01,
                       n_estimators = 1000, objective = 'binary')

#### XGBoost


In [13]:
model = XGBClassifier()

#### SVC

In [32]:
model = SVC()

#### Knn

In [33]:
model = KNeighborsClassifier()

## Parallel Process




In [42]:
kfolds = StratifiedKFold(n_splits=5, shuffle=True)
n_jobs = -1

st = time()

if n_jobs == 1:
    print('Running single process loop')
    res = []

    for train_index, valid_index in tqdm(kfolds.split(X_train, Y_train)):
        res.append(fit_and_score(clone(model), train_index, valid_index, X_train, Y_train))
else:
    print('Running parallel loop')
    parallel = Parallel(n_jobs=n_jobs, verbose=1)
    res = parallel(delayed(fit_and_score)(clone(model), train_idx, valid_idx, X_train, Y_train)
              for train_idx, valid_idx in kfolds.split(X_train, Y_train))


end = time()

print('Total time taken ', (end - st)/60, ' mins')

Running parallel loop


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Total time taken  0.0039035360018412272  mins


[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    0.2s finished


## Metrics

In [43]:
res = pd.DataFrame(res, columns=['ROC_train', 'ROC_val', 'F1_train', 'F1_val', 'MCC_train', 'MCC_val'])
res

,ROC_train,ROC_val,F1_train,F1_val,MCC_train,MCC_val
0,0.500000,0.500000,0.000000,0.00000,0.000000,0.000000
1,0.500000,0.500000,0.000000,0.00000,0.000000,0.000000
2,0.500000,0.500000,0.000000,0.00000,0.000000,0.000000
3,0.500000,0.500000,0.000000,0.00000,0.000000,0.000000
4,0.592174,0.547176,0.573193,0.53202,0.148667,0.062656


In [ ]:
print('Mean roc auc (train, valid) : ', res['ROC_train'].mean(), ', ', res['ROC_val'].mean())

## Training on Entire dataset

In [44]:
model.fit(X_train, Y_train)

LogisticRegression(C=0.01, penalty='l1', solver='liblinear')

## Model Performance

In [45]:
# model_pipe.roc_auc_score(X_train,Y_train)
Y_train_preds = model.predict_proba(X_train)[:,1]
train_score = roc_auc_score(Y_train,Y_train_preds)

Y_test_preds = model.predict_proba(X_test)[:,1]

print('Training score: ', train_score)
print(classification_report(Y_train, model.predict(X_train)))

Training score:  0.5830370337854645
              precision    recall  f1-score   support

           0       0.55      0.58      0.57       981
           1       0.58      0.55      0.56      1045

    accuracy                           0.57      2026
   macro avg       0.57      0.57      0.57      2026
weighted avg       0.57      0.57      0.57      2026

